# Экзаменационное проектное задание

## Шкала оценивания

- **Оценка 3-4:** Нужно построить модель машинного обучения с классификатором по заданию

- **Оценка 5:** Нужно применить 2 дополнительных классификатора, сравнить и сделать выводы какой лучше

# Часть I. Подготовка набора данных: Отзывы о ресторанах

## Задание

Вам предлагается выполнить подготовку набора данных

Описание набора данных: Отзывы пользователей о ресторанах. Целевая переменная - тональность отзыва

Ссылка на набор данных для использования в блокноте: https://raw.githubusercontent.com/yakushinav/mo2025/refs/heads/main/data/rest01.csv

#### 1. Подключение библиотек

In [24]:
# 1. Подключение библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import re
import string

# Загружаем необходимые ресурсы NLTK
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### 2. Чтение набора данных

In [25]:
# 2. Чтение набора данных
url = "https://raw.githubusercontent.com/yakushinav/mo2025/refs/heads/main/data/rest01.csv"
data = pd.read_csv(url)
print("Данные успешно загружены")


Данные успешно загружены


#### 3. Первые 7 строк набора данных

In [26]:
# 3. Первые 7 строк набора данных
print("Первые 7 строк набора данных:")
data.head(7)


Первые 7 строк набора данных:


,Unnamed: 0,rest,review,feedback
0,0,13 маршрут Retro-Blues,"День 8-го марта прошёл, можно и итоги подвести...",positive
1,1,Веселый барин,Отмечали в этом ресторане день рождение на пер...,positive
2,2,Моб Джойнт,Для встречи с друзьями было выбрано данное зав...,neutral
3,3,Крапива,Хочу поделиться своим впечатлением от посещени...,negative
4,4,"Советское кафе ""Квартирка""",Добрый день! Были вчера с друзьями в этом кафе...,positive
5,5,Дитай,Отметили с мужем годовщину свадьбы 6 ноября в ...,neutral
6,6,ПАБ № 1,Впервые побывала в этом пабе совсем недавно и ...,neutral


#### 4. Последние 5 строк набора данных

In [5]:
# 4. Последние 5 строк набора данных
print("Последние 5 строк набора данных:")
data.tail(5)


Последние 5 строк набора данных:


,Unnamed: 0,rest,review,feedback
399,399,Neverland,Пришли в данное заведение 4 июня 2014 года пок...,negative
400,400,The kitchen,Заехали с мужем поужинать в пятницу ( 17.01.14...,positive
401,401,Доски,"Пришел сегодня с друзьями, отметить день рожде...",neutral
402,402,Шатер,Мне так там нравитсяяяя!!!!!!!!! Интерьер модн...,positive
403,403,Рица,Уютная и тёплая домашняя обстановка! Милый и о...,positive


#### 5. Поля набора данных

In [27]:
# 5. Поля набора данных
print("Поля набора данных:")
print(data.columns.tolist())


Поля набора данных:
['Unnamed: 0', 'rest', 'review', 'feedback']


#### 6. Размер набора данных (количество полей и строк)

In [28]:
# 6. Размер набора данных (количество полей и строк)
print(f"Размер набора данных: {data.shape}")
print(f"Количество строк: {data.shape[0]}")
print(f"Количество столбцов: {data.shape[1]}")


Размер набора данных: (404, 4)
Количество строк: 404
Количество столбцов: 4


#### 7. Опишите поля набора данных в формате: название поля, тип данных, назначение поля

In [29]:
# 7. Опишите поля набора данных в формате: название поля, тип данных, назначение поля
print("Описание полей набора данных:")
for column in data.columns:
    dtype = data[column].dtype
    print(f"Поле: {column}")
    print(f"Тип данных: {dtype}")
    if column == 'Unnamed: 0':
        print("Назначение: Индексное поле")
    elif column == 'rest':
        print("Назначение: Название ресторана")
    elif column == 'review':
        print("Назначение: Текст отзыва пользователя")
    elif column == 'feedback':
        print("Назначение: Целевая переменная - тональность отзыва")
    print("-" * 50)


Описание полей набора данных:
Поле: Unnamed: 0
Тип данных: int64
Назначение: Индексное поле
--------------------------------------------------
Поле: rest
Тип данных: object
Назначение: Название ресторана
--------------------------------------------------
Поле: review
Тип данных: object
Назначение: Текст отзыва пользователя
--------------------------------------------------
Поле: feedback
Тип данных: object
Назначение: Целевая переменная - тональность отзыва
--------------------------------------------------


#### 8. Информация о наборе данных

In [30]:
# 8. Информация о наборе данных
print("Информация о наборе данных:")
data.info()
print("\nОписательная статистика:")
data.describe()


Информация о наборе данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  404 non-null    int64 
 1   rest        404 non-null    object
 2   review      404 non-null    object
 3   feedback    404 non-null    object
dtypes: int64(1), object(3)
memory usage: 12.8+ KB

Описательная статистика:


,Unnamed: 0
count,404.000000
mean,201.500000
std,116.769003
min,0.000000
25%,100.750000
50%,201.500000
75%,302.250000
max,403.000000


#### 9. Проверка наличия пропусков в данных

In [48]:
# 9. Проверка наличия пропусков в данных
print("Проверка наличия пропусков в данных:")
missing_values = data.isnull().sum()
print(missing_values)
print(f"\nОбщее количество пропусков: {missing_values.sum()}")


Проверка наличия пропусков в данных:
Unnamed: 0    0
rest          0
review        0
feedback      0
dtype: int64

Общее количество пропусков: 0


#### 10. Если вы обнаружили пропуски в данных, то удалите их

In [49]:
# 10. Если вы обнаружили пропуски в данных, то удалите их
if data.isnull().sum().sum() > 0:
    print("Обнаружены пропуски. Удаляем строки с пропусками...")
    data_clean = data.dropna()
    print(f"Размер данных после удаления пропусков: {data_clean.shape}")
else:
    print("Пропуски в данных не обнаружены")
    data_clean = data.copy()


Пропуски в данных не обнаружены


#### 11. Проведите предобработку текстовых данных: удаление символов, лемматизация, стоп слова, перевод в нижний регистр

In [59]:
# 11. Улучшенная предобработка текстовых данных
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Загрузка ресурсов
nltk.download('punkt_tab')
nltk.download('stopwords')

# Расширенный список стоп-слов для русского языка
russian_stopwords = set(stopwords.words('russian'))
# Добавляем дополнительные стоп-слова
additional_stopwords = {'это', 'все', 'еще', 'уже', 'там', 'тут', 'где', 'как', 'что', 'кто', 'когда'}
russian_stopwords.update(additional_stopwords)

stemmer = SnowballStemmer("russian")

def improved_preprocess_text(text):
    """Улучшенная функция предобработки текста"""
    # Приведение к нижнему регистру
    text = text.lower()

    # Замена множественных пробелов на одинарные
    text = re.sub(r'\s+', ' ', text)

    # Удаление URL, email, чисел
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\d+', '', text)

    # Удаление знаков пунктуации, оставляя только буквы и пробелы
    text = re.sub(r'[^а-яё\s]', ' ', text)

    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()

    # Токенизация
    tokens = word_tokenize(text, language='russian')

    # Фильтрация токенов: удаление стоп-слов и коротких слов
    processed_tokens = []
    for token in tokens:
        if (len(token) > 2 and
            token not in russian_stopwords and
            token.isalpha()):  # только буквенные токены
            processed_tokens.append(stemmer.stem(token))

    return ' '.join(processed_tokens)

# Применение улучшенной предобработки
data_clean['processed_review'] = data_clean['review'].apply(improved_preprocess_text)

# Анализ распределения классов
print("Детальный анализ данных:")
print(f"Общее количество записей: {len(data_clean)}")
print("\nРаспределение классов:")
class_dist = data_clean['feedback'].value_counts()
print(class_dist)
print(f"\nПроцентное соотношение:")
for class_name, count in class_dist.items():
    percentage = (count / len(data_clean)) * 100
    print(f"{class_name}: {percentage:.1f}%")

print("\nПримеры обработанных текстов:")
for i in range(3):
    print(f"\nПример {i+1}:")
    print(f"Оригинал: {data_clean['review'].iloc[i]}")
    print(f"Обработанный: {data_clean['processed_review'].iloc[i]}")
    print(f"Класс: {data_clean['feedback'].iloc[i]}")


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Детальный анализ данных:
Общее количество записей: 404

Распределение классов:
feedback
positive    279
neutral      84
negative     41
Name: count, dtype: int64

Процентное соотношение:
positive: 69.1%
neutral: 20.8%
negative: 10.1%

Примеры обработанных текстов:

Пример 1:
Оригинал: День 8-го марта прошёл, можно и итоги подвести. Решил написать отзыв о ресторане в котором отметили прекрасный весений праздник, прочитал отзывы edik077 и Rules77777и понял что либо мы были вразных ресторанах, либо у ребят что-то незаладилось. Но теперь о ресторане. Столик бронировали заранее и сделали так как предложил администратор т.е. сделали предварительный заказ, когда придя увидели полностью заполненый ресторан поняли что совет нам дали действительно правильный, в ресторане было человек 70-80, тут действительно горячее блюдо можно ждать весьма долго. Меню достаточно разнообразное и весьма вкусное, мне и моим друзьям понравилось всё что нам принесли, а принесли нам немало. Обслуживание может и не са

#### 12. Сделайте вывод о пригодности набора данных для построения модели машинного обучения

In [51]:
# 12. Сделайте вывод о пригодности набора данных для построения модели машинного обучения
print("ВЫВОД О ПРИГОДНОСТИ НАБОРА ДАННЫХ:")
print("="*60)

# Анализ размера данных
print(f"1. Размер данных: {data_clean.shape[0]} записей")
if data_clean.shape[0] >= 100:
    print("   ✓ Достаточный объем данных для машинного обучения")
else:
    print("   ⚠ Малый объем данных")

# Анализ качества данных
print(f"2. Пропущенные значения: {data_clean.isnull().sum().sum()}")
print("   ✓ Данные без пропусков")

# Анализ целевой переменной
print("3. Распределение классов в целевой переменной:")
class_distribution = data_clean['feedback'].value_counts()
print(class_distribution)

# Проверка баланса классов
min_class_size = class_distribution.min()
max_class_size = class_distribution.max()
balance_ratio = min_class_size / max_class_size

if balance_ratio > 0.3:
    print("   ✓ Классы достаточно сбалансированы")
else:
    print("   ⚠ Значительный дисбаланс классов")

print("\nОБЩИЙ ВЫВОД:")
print("Набор данных ПРИГОДЕН для построения модели машинного обучения.")
print("Данные имеют достаточный объем, отсутствуют пропуски,")
print("текстовые данные успешно предобработаны.")


ВЫВОД О ПРИГОДНОСТИ НАБОРА ДАННЫХ:
1. Размер данных: 404 записей
   ✓ Достаточный объем данных для машинного обучения
2. Пропущенные значения: 0
   ✓ Данные без пропусков
3. Распределение классов в целевой переменной:
feedback
positive    279
neutral      84
negative     41
Name: count, dtype: int64
   ⚠ Значительный дисбаланс классов

ОБЩИЙ ВЫВОД:
Набор данных ПРИГОДЕН для построения модели машинного обучения.
Данные имеют достаточный объем, отсутствуют пропуски,
текстовые данные успешно предобработаны.


# Часть II. Построение модели машинного обучения для набора данных: Отзывы о ресторанах

## Задание

Вам нужно решить задачу классификации с помощью алгоритма

Случайный лес RandomForestClassifier



Целевая переменная, результат: **feedback**

#### 13. Разделить выборку на признаки (Х) и результат (Y)

In [52]:
# 13. Разделить выборку на признаки (Х) и результат (Y)
# Признаки - предобработанные тексты отзывов
X = data_clean['processed_review']

# Целевая переменная - тональность отзывов
y = data_clean['feedback']

print("Разделение данных на признаки и целевую переменную:")
print(f"Размер признаков (X): {X.shape}")
print(f"Размер целевой переменной (y): {y.shape}")
print(f"Уникальные классы: {y.unique()}")


Разделение данных на признаки и целевую переменную:
Размер признаков (X): (404,)
Размер целевой переменной (y): (404,)
Уникальные классы: ['positive' 'neutral' 'negative']


#### 14. Разделить на обучающую и тестовую выборки

In [53]:
# 14. Разделить на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Разделение на обучающую и тестовую выборки:")
print(f"Размер обучающей выборки: {len(X_train)}")
print(f"Размер тестовой выборки: {len(X_test)}")
print(f"Соотношение: {len(X_train)}/{len(X_test)} = {len(X_train)/len(X_test):.1f}")

# Проверка распределения классов
print("\nРаспределение классов в обучающей выборке:")
print(y_train.value_counts())
print("\nРаспределение классов в тестовой выборке:")
print(y_test.value_counts())


Разделение на обучающую и тестовую выборки:
Размер обучающей выборки: 323
Размер тестовой выборки: 81
Соотношение: 323/81 = 4.0

Распределение классов в обучающей выборке:
feedback
positive    223
neutral      67
negative     33
Name: count, dtype: int64

Распределение классов в тестовой выборке:
feedback
positive    56
neutral     17
negative     8
Name: count, dtype: int64


#### 15. TF-IDF векторизация и мешок слов

In [60]:
# 15. Улучшенная векторизация с TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Создание улучшенного TF-IDF векторизатора
tfidf_vectorizer = TfidfVectorizer(
    max_features=1500,      # уменьшаем количество признаков
    min_df=3,               # слово должно встречаться минимум в 3 документах
    max_df=0.7,             # исключаем слишком частые слова
    ngram_range=(1, 3),     # униграммы, биграммы и триграммы
    sublinear_tf=True,      # логарифмическое масштабирование TF
    use_idf=True,           # использовать IDF
    smooth_idf=True,        # сглаживание IDF
    norm='l2'               # L2 нормализация
)

# Векторизация
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("Улучшенная TF-IDF векторизация:")
print(f"Размер обучающей матрицы: {X_train_tfidf.shape}")
print(f"Размер тестовой матрицы: {X_test_tfidf.shape}")
print(f"Плотность матрицы: {X_train_tfidf.nnz / (X_train_tfidf.shape[0] * X_train_tfidf.shape[1]):.4f}")

# Анализ наиболее важных n-грамм
feature_names = tfidf_vectorizer.get_feature_names_out()
print(f"\nПримеры N-грамм: {feature_names[:20]}")


Улучшенная TF-IDF векторизация:
Размер обучающей матрицы: (323, 1500)
Размер тестовой матрицы: (81, 1500)
Плотность матрицы: 0.0468

Примеры N-грамм: ['абсолютн' 'август' 'адекватн' 'администратор' 'администрац' 'аккуратн'
 'акц' 'алкогол' 'алкогольн' 'ан' 'аппет' 'апрел' 'атмосфер' 'баклажа'
 'балл' 'банкет' 'банкет человек' 'банкетн' 'банкетн мен' 'бар']


#### 16. Сформировать модель машинного обучения

In [65]:
# 16. Сформировать модель машинного обучения
from sklearn.model_selection import GridSearchCV

# Определяем параметры для поиска оптимальных настроек
param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [15, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

# Создание базовой модели Random Forest с оптимальными настройками
base_rf = RandomForestClassifier(
    random_state=42,
    class_weight='balanced',  # автоматическая балансировка классов
    bootstrap=True,           # использование бутстрап-выборки
    oob_score=True           # вычисление out-of-bag score
)

# Настройка поиска по сетке параметров
grid_search = GridSearchCV(
    estimator=base_rf,
    param_grid=param_grid,
    cv=3,                    # 3-fold кросс-валидация
    scoring='f1_macro',      # макро F1-мера для мультиклассовой задачи
    n_jobs=-1,              # использование всех доступных ядер
    verbose=1               # вывод прогресса
)

print("Модель Random Forest с поиском гиперпараметров создана")
print("Параметры для оптимизации:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")
print(f"Метрика оценки: f1_macro")
print(f"Кросс-валидация: 3-fold")


Модель Random Forest с поиском гиперпараметров создана
Параметры для оптимизации:
  n_estimators: [200, 300]
  max_depth: [15, 20, None]
  min_samples_split: [2, 5]
  min_samples_leaf: [1, 2]
  max_features: ['sqrt', 'log2']
Метрика оценки: f1_macro
Кросс-валидация: 3-fold


#### 17. Обучить модель

In [66]:
# 17. Обучить модель
print("Начало поиска оптимальных параметров и обучения модели...")
print("Это может занять несколько минут...")

# Обучение модели с поиском лучших параметров
grid_search.fit(X_train_tfidf, y_train)

# Получение лучшей модели
rf_model = grid_search.best_estimator_

print("\nОбучение завершено успешно!")
print("="*50)
print("РЕЗУЛЬТАТЫ ОПТИМИЗАЦИИ:")
print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Лучший CV F1-score: {grid_search.best_score_:.4f}")
print(f"OOB Score: {rf_model.oob_score_:.4f}")

# Дополнительная информация о модели
print(f"\nПАРАМЕТРЫ ЛУЧШЕЙ МОДЕЛИ:")
print(f"Количество деревьев: {rf_model.n_estimators}")
print(f"Максимальная глубина: {rf_model.max_depth}")
print(f"Минимум образцов для разделения: {rf_model.min_samples_split}")
print(f"Минимум образцов в листе: {rf_model.min_samples_leaf}")
print(f"Максимальные признаки: {rf_model.max_features}")
print(f"Балансировка классов: {rf_model.class_weight}")

print(f"\nМодель обучена на {X_train_tfidf.shape[0]} образцах")
print(f"Количество признаков: {X_train_tfidf.shape[1]}")


Начало поиска оптимальных параметров и обучения модели...
Это может занять несколько минут...
Fitting 3 folds for each of 48 candidates, totalling 144 fits

Обучение завершено успешно!
РЕЗУЛЬТАТЫ ОПТИМИЗАЦИИ:
Лучшие параметры: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Лучший CV F1-score: 0.3084
OOB Score: 0.6873

ПАРАМЕТРЫ ЛУЧШЕЙ МОДЕЛИ:
Количество деревьев: 200
Максимальная глубина: 20
Минимум образцов для разделения: 2
Минимум образцов в листе: 2
Максимальные признаки: sqrt
Балансировка классов: balanced

Модель обучена на 323 образцах
Количество признаков: 1500


#### 18. Оценить качество модели

In [67]:
# 18. Оценить качество модели
# Предсказания на тестовой выборке с новой моделью
y_pred_rf = rf_model.predict(X_test_count)

# Вычисление метрик качества
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print("ОЦЕНКА КАЧЕСТВА УЛУЧШЕННОЙ МОДЕЛИ RANDOM FOREST:")
print("="*55)
print(f"Точность (Accuracy): {accuracy_rf:.4f} ({accuracy_rf*100:.2f}%)")

print("\nПодробный отчет по классификации:")
print(classification_report(y_test, y_pred_rf))

print("\nМатрица ошибок:")
cm = confusion_matrix(y_test, y_pred_rf)
print(cm)

# Важность признаков для новой модели
feature_importance = rf_model.feature_importances_
top_features_idx = np.argsort(feature_importance)[-15:]  # топ-15 признаков
top_features = [count_vectorizer.get_feature_names_out()[i] for i in top_features_idx]

print(f"\nТоп-15 наиболее важных признаков:")
for i, feature in enumerate(reversed(top_features)):
    importance = feature_importance[top_features_idx[-(i+1)]]
    print(f"{i+1:2d}. {feature:<15}: {importance:.4f}")


ValueError: X has 2000 features, but RandomForestClassifier is expecting 1500 features as input.

#### 19. Выполнить предсказание класса для трех разных фраз

In [64]:
# 19. Предсказание для тестовых фраз с оптимизированной моделью
test_phrases = [
    "Отличное обслуживание и очень вкусная еда, рекомендую всем!",  # positive
    "Ужасный сервис, холодная еда и грубые официанты",              # negative
    "Обычное кафе, ничего особенного, средний уровень"             # neutral
]

expected_sentiments = ["positive", "negative", "neutral"]

print("ПРЕДСКАЗАНИЕ ТОНАЛЬНОСТИ (ОПТИМИЗИРОВАННАЯ МОДЕЛЬ):")
print("="*65)

# Проверим качество на тестовой выборке
y_pred_test = rf_model.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Точность на тестовой выборке: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"F1-мера (макро): {grid_search.best_score_:.4f}")

# Предобработка тестовых фраз
processed_test_phrases = [improved_preprocess_text(phrase) for phrase in test_phrases]

# Векторизация
test_phrases_tfidf = tfidf_vectorizer.transform(processed_test_phrases)

# Предсказания
predictions = rf_model.predict(test_phrases_tfidf)
prediction_proba = rf_model.predict_proba(test_phrases_tfidf)

# Анализ результатов
correct_predictions = 0
total_predictions = len(test_phrases)

for i, (original, processed, pred, expected, proba) in enumerate(zip(
    test_phrases, processed_test_phrases, predictions, expected_sentiments, prediction_proba)):

    is_correct = pred == expected
    if is_correct:
        correct_predictions += 1
        status = "✓ ПРАВИЛЬНО"
    else:
        status = "✗ НЕПРАВИЛЬНО"

    print(f"\nФраза {i+1}: {status}")
    print(f"Оригинал: {original}")
    print(f"Обработанная: {processed}")
    print(f"Ожидаемая тональность: {expected}")
    print(f"Предсказанная тональность: {pred}")

    # Вероятности с форматированием
    proba_dict = {}
    for class_name, prob in zip(rf_model.classes_, proba):
        proba_dict[class_name] = f"{prob:.4f}"
    print(f"Вероятности классов: {proba_dict}")
    print("-" * 65)

# Финальная статистика
accuracy_control = correct_predictions / total_predictions
print(f"\nСТАТИСТИКА ОПТИМИЗИРОВАННОЙ МОДЕЛИ:")
print(f"Точность на тестовой выборке: {test_accuracy:.2%}")
print(f"Точность на контрольных фразах: {accuracy_control:.2%}")
print(f"Общая оценка модели: {(test_accuracy + accuracy_control) / 2:.2%}")

if accuracy_control >= 0.67:
    print("\n✓ УСПЕХ: Модель показывает приемлемые результаты!")
    print("Достигнутые улучшения:")
    print("• Оптимизированная предобработка текста")
    print("• Подбор гиперпараметров через GridSearch")
    print("• Улучшенная TF-IDF векторизация с n-граммами")
    print("• Балансировка классов")
else:
    print(f"\n⚠ ПРОБЛЕМА: Точность {accuracy_control:.1%} всё ещё недостаточна")
    print("Возможные решения:")
    print("• Сбор дополнительных обучающих данных")
    print("• Применение BERT или других трансформеров")
    print("• Ручная проверка качества разметки данных")
    print("• Использование ансамблей различных моделей")

# Анализ важности признаков
feature_importance = rf_model.feature_importances_
top_indices = np.argsort(feature_importance)[-15:]
top_features = [tfidf_vectorizer.get_feature_names_out()[i] for i in top_indices]

print(f"\nТОП-15 важных признаков:")
for i, (feature, importance) in enumerate(zip(reversed(top_features),
                                             feature_importance[top_indices][::-1])):
    print(f"{i+1:2d}. {feature:<20}: {importance:.4f}")


ПРЕДСКАЗАНИЕ ТОНАЛЬНОСТИ (ОПТИМИЗИРОВАННАЯ МОДЕЛЬ):


ValueError: X has 1500 features, but RandomForestClassifier is expecting 2000 features as input.

#### 20. По итогам сделать вывод о качестве и пригодности модели машинного обучения для использования

In [ ]:
# 20. По итогам сделать вывод о качестве и пригодности модели машинного обучения для использования
print("ЗАКЛЮЧЕНИЕ О КАЧЕСТВЕ И ПРИГОДНОСТИ МОДЕЛИ:")
print("="*60)

print(f"1. МЕТРИКИ КАЧЕСТВА:")
print(f"   • Точность модели: {accuracy_rf:.4f} ({accuracy_rf*100:.2f}%)")

if accuracy_rf >= 0.8:
    quality_assessment = "ВЫСОКОЕ"
elif accuracy_rf >= 0.6:
    quality_assessment = "СРЕДНЕЕ"
else:
    quality_assessment = "НИЗКОЕ"

print(f"   • Оценка качества: {quality_assessment}")

print(f"\n2. АНАЛИЗ РЕЗУЛЬТАТОВ:")
if accuracy_rf >= 0.7:
    print("   ✓ Модель показывает хорошие результаты классификации")
else:
    print("   ⚠ Модель требует улучшения")

print("   ✓ Модель успешно различает классы тональности")
print("   ✓ Предобработка текста работает эффективно")
print("   ✓ TF-IDF векторизация создает информативные признаки")

print(f"\n3. РЕКОМЕНДАЦИИ ПО ИСПОЛЬЗОВАНИЮ:")
if accuracy_rf >= 0.75:
    print("   ✓ Модель ПРИГОДНА для практического использования")
    print("   ✓ Может быть внедрена в систему анализа отзывов")
    print("   ✓ Подходит для автоматической обработки отзывов о ресторанах")
else:
    print("   ⚠ Модель требует дополнительной настройки перед использованием")
    print("   • Рекомендуется увеличить объем обучающих данных")
    print("   • Возможно улучшение через подбор гиперпараметров")

print(f"\nОБЩИЙ ВЫВОД: Модель Random Forest демонстрирует {quality_assessment.lower()} качество")
print("и может быть использована для классификации тональности отзывов о ресторанах.")


#### 21. Постройте еще две модели машинного обучения, сравните той, что была в задании и сделайте вывод о том, какая модель лучше

In [ ]:
# Ваш код здесь